In [9]:
import h2o
import pandas as pd
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,8 mins 47 secs
H2O_cluster_timezone:,America/Chicago
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,2 months and 27 days
H2O_cluster_name:,H2O_from_python_alonm_rd3dj4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.404 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


In [10]:
df = pd.read_csv('final_dataframe.csv')
df = df.drop(columns=['district'])

df['Native People'] = df['Native People'].astype(str).str.replace(',', '').astype(int)
df['Unemployment Rate'] = df['Unemployment Rate'].astype(str).str.replace(',', '').astype(float)
df['Mean household income'] = df['Mean household income'].astype(str).str.replace(',', '').astype(int)
df['Families income below poverty'] = df['Families income below poverty'].astype(str).str.replace(',', '').astype(float)
df['Graduate or professional degree'] = df['Graduate or professional degree'].astype(str).str.replace(',', '').astype(float)
df['Hispanic'] = df['Hispanic'].astype(str).str.replace(',', '').astype(int)
df['Male'] = df['Male'].astype(str).str.replace(',', '').astype(int)
df['Female'] = df['Female'].astype(str).str.replace(',', '').astype(int)

df['presidential_votes_ratio'] = df['presidential_total_votes'] / (df['18 years and over'])
response = 'presidential_votes_ratio'
predictors = ['Native People', 'Unemployment Rate', 'Mean household income', 'Families income below poverty', 'Graduate or professional degree', 'Hispanic', 'Male', 'Female']

df.head()

h2o_df = h2o.H2OFrame(df)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [11]:
import os

input_path = 'presidential_turnout_autoML'
file_names = os.listdir(input_path)
for i, file_name in enumerate(file_names):
    model_path = os.path.join(input_path, file_name)
    model = h2o.load_model(model_path)
    print(f'Model {i} loaded.')
    model_summary = model.summary()
    print(model_summary)

Model 0 loaded.
Status of Neuron Layers: predicting presidential_votes_ratio, regression, gaussian distribution, Quadratic loss, 501 weights/biases, 10,5 KB, 360.360 training samples, mini-batch size 1
    layer    units    type              dropout    l1    l2    mean_rate             rate_rms              momentum    mean_weight           weight_rms          mean_bias            bias_rms
--  -------  -------  ----------------  ---------  ----  ----  --------------------  --------------------  ----------  --------------------  ------------------  -------------------  -----------------------
    1        8        Input             0.0
    2        50       RectifierDropout  40.0       0.0   0.0   0.014593527158140204  0.029078200459480286  0.0         -0.03609370522317477  0.489656925201416   -0.8225816972471045  0.2532920837402344
    3        1        Linear                       0.0   0.0   0.009569003623910248  0.01558372750878334   0.0         0.3735393318533897    0.4319138526916

In [12]:
from h2o.frame import H2OFrame

# Create an empty list to store RMSE values and model names
model_rmse_list = []

# Iterate through the models and calculate RMSE
for file_name in file_names:
    model_path = os.path.join(input_path, file_name)
    model = h2o.load_model(model_path)
    
    # Assuming you have a test dataset loaded as 'test_data'
    predictions = model.predict(h2o_df)
    rmse = model.model_performance(h2o_df).rmse()
    
    model_rmse_list.append((file_name, rmse))

# Sort the models by RMSE
sorted_models = sorted(model_rmse_list, key=lambda x: x[1])

# Get the best model (the one with the lowest RMSE)
best_model_name = sorted_models[0][0]
best_model_path = os.path.join(input_path, best_model_name)
best_model = h2o.load_model(best_model_path)

# Get the architecture of the best model
best_model_architecture = best_model.algo
print(f'The architecture of the best model ({best_model_name}) is: {best_model_architecture}')# Get the best model (the one with the lowest RMSE)
best_model_name = sorted_models[0][0]
print(f'Model {best_model} loaded.')
model_summary = best_model.summary()
print(model_summary)

deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
deeplearning prediction progress: |██████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
The architecture of the best model (DeepLearning_g